This file contains all definitions needed for the scenario generator.

Please call the 'generate_scenario' function to get a new, random scenario. Note you can enter a custom random seed.

Please note that scenarios are generated using the following logic:
1. Under the 'Default values' markdown, you can see the expected number of passengers for evening rush hour, for any given workday.
2. This expected count across two hours is broken down into all of its five minute intervals (24 intervals total)
3. If E is the expected count for any line, we then have an expected E/24 count for each interval.
4. We could then generate a count for each inteval and line, so that it follows a Normal distribution with mean E/24 and std. dev 0.15*(E/24)
5. Since Normal could return negative, get max(count, 0), meaning nobody showed up to take the line (should happen rarely)
6. Then, we floor the returned value, as Normal can return non integer.

In [152]:
import numpy as np
from typing import Dict, Tuple
from time import time_ns

Default values

In [153]:
EXP_RIDERSHIP: Dict[str, int] = {
    'Lakeshore West': 33239,
    'Lakeshore East': 25769,
    'Kitchener': 15071,
    'Milton': 13138,
    'Barrie': 8636,
    'Stouffville': 7341,
    'Richmond Hill': 4642
}

SPREAD = 0.15 #Modify this to get scenarios further from the exp values above

Function for scenarios

In [154]:
def generate_scenario(seed: int | str = None) -> Dict[str, Dict[str, int]]:
    """
    Generates ridership demand scenario.

    Returns dict of all time interval and line combinations.

    seed: int | str => can be used for replication purposes. Random seed to use.
    """

    if seed is None: seed = time_ns()

    if type(seed) is not int: seed = sum([ord(char) for char in seed])

    np.random.seed(seed%(2**32-1))

    scenario: Dict[str, Dict[str, int]] = {}

    #Initializing dicts
    for line in EXP_RIDERSHIP.keys():
        scenario[line] = {}

    #Iterating lines
    for line, exp_demand in EXP_RIDERSHIP.items():
        mu = exp_demand/24
        sigma = mu*SPREAD

        #Iterating time intervals
        for t in range(0, 120, 5):
            demand = np.random.normal(loc=mu, scale=sigma)
            scenario[line][f"{4+int(t/60)}:{(t-60*int(t/60)):02}"] = max(0, int(demand))

    return scenario



In [155]:
generate_scenario("MIE562")

{'Lakeshore West': {'4:00': 1656,
  '4:05': 1297,
  '4:10': 1178,
  '4:15': 1631,
  '4:20': 1465,
  '4:25': 1700,
  '4:30': 1175,
  '4:35': 1654,
  '4:40': 1449,
  '4:45': 1522,
  '4:50': 1440,
  '4:55': 1399,
  '5:00': 1540,
  '5:05': 1236,
  '5:10': 1408,
  '5:15': 1102,
  '5:20': 1554,
  '5:25': 1396,
  '5:30': 1500,
  '5:35': 1399,
  '5:40': 1424,
  '5:45': 1879,
  '5:50': 1153,
  '5:55': 1543},
 'Lakeshore East': {'4:00': 1267,
  '4:05': 1111,
  '4:10': 1098,
  '4:15': 1400,
  '4:20': 832,
  '4:25': 1018,
  '4:30': 701,
  '4:35': 1232,
  '4:40': 955,
  '4:45': 1302,
  '4:50': 920,
  '4:55': 1276,
  '5:00': 814,
  '5:05': 1052,
  '5:10': 1126,
  '5:15': 1074,
  '5:20': 911,
  '5:25': 1116,
  '5:30': 1100,
  '5:35': 1040,
  '5:40': 971,
  '5:45': 1315,
  '5:50': 827,
  '5:55': 870},
 'Kitchener': {'4:00': 657,
  '4:05': 607,
  '4:10': 471,
  '4:15': 655,
  '4:20': 620,
  '4:25': 604,
  '4:30': 594,
  '4:35': 798,
  '4:40': 732,
  '4:45': 697,
  '4:50': 764,
  '4:55': 560,
  '5:00': 

In [156]:
generate_scenario()

{'Lakeshore West': {'4:00': 1494,
  '4:05': 1346,
  '4:10': 1673,
  '4:15': 1202,
  '4:20': 1521,
  '4:25': 1427,
  '4:30': 1301,
  '4:35': 1156,
  '4:40': 1335,
  '4:45': 1715,
  '4:50': 1423,
  '4:55': 1698,
  '5:00': 1322,
  '5:05': 1750,
  '5:10': 1590,
  '5:15': 1233,
  '5:20': 1635,
  '5:25': 1605,
  '5:30': 1069,
  '5:35': 1257,
  '5:40': 1631,
  '5:45': 1151,
  '5:50': 1176,
  '5:55': 1420},
 'Lakeshore East': {'4:00': 1013,
  '4:05': 1142,
  '4:10': 1320,
  '4:15': 1162,
  '4:20': 1027,
  '4:25': 1203,
  '4:30': 1051,
  '4:35': 1090,
  '4:40': 1048,
  '4:45': 1013,
  '4:50': 1031,
  '4:55': 935,
  '5:00': 1235,
  '5:05': 1126,
  '5:10': 884,
  '5:15': 1002,
  '5:20': 1257,
  '5:25': 1160,
  '5:30': 1186,
  '5:35': 989,
  '5:40': 1010,
  '5:45': 1345,
  '5:50': 978,
  '5:55': 989},
 'Kitchener': {'4:00': 714,
  '4:05': 588,
  '4:10': 553,
  '4:15': 655,
  '4:20': 438,
  '4:25': 603,
  '4:30': 643,
  '4:35': 614,
  '4:40': 690,
  '4:45': 627,
  '4:50': 678,
  '4:55': 680,
  '5:0